In [ ]:
import pandas as pdall_pr_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_pull_request.parquet")all_repo_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_repository.parquet")all_user_df = pd.read_parquet("hf://datasets/hao-li/AIDev/all_user.parquet")# Ensure created_at is datetimeall_pr_df['created_at'] = pd.to_datetime(all_pr_df['created_at'])# Count agent PRs per repopr_counts = all_pr_df.groupby('repo_id').size().reset_index(name='agent_pr_count')# Filter for repos with >= 100 agent PRslarge_repos = pr_counts[pr_counts['agent_pr_count'] >= 100]# Get first agent PR date per repofirst_pr_dates = all_pr_df.groupby('repo_id')['created_at'].min().reset_index(name='first_pr_date')# Merge counts and first dateresult = large_repos.merge(first_pr_dates, on='repo_id')# Merge with repo metadata for repo name and URLresult = result.merge(all_repo_df, left_on='repo_id', right_on='id')# Select relevant columnsresult = result[['full_name', 'url', 'agent_pr_count', 'first_pr_date']]# Sort by number of agent PRs descendingresult = result.sort_values(by='agent_pr_count', ascending=False)# Save to TXTwith open("top_agent_repos.txt", "w", encoding="utf-8") as f:    for idx, row in result.iterrows():        f.write(f"{row['full_name']} | {row['url']} | {row['agent_pr_count']} agent PRs | first PR: {row['first_pr_date'].date()}")print("Saved top 100 agent repos to top_agent_repos.txt")